In [1]:
# from pyspark.sql import SparkSession
# from pyspark.sql.functions import col, udf
# from pyspark.sql.types import StringType, FloatType
import spacy
from textblob import TextBlob
import plotly.express as px
import pandas as pd

In [2]:
# spark = SparkSession.builder.appName("JSONL to PySpark DataFrame").config("spark.sql.caseSensitive", "true").getOrCreate()
file_path = "../data/All_Beauty.jsonl"
# df_reviews = spark.read.json(file_path)
df_reviews = pd.read_json(file_path, lines=True)
file_path = "../data/meta_All_Beauty.jsonl"
# df_meta = spark.read.json(file_path)
df_meta = pd.read_json(file_path, lines=True)

In [3]:
# df = df_reviews.join(df_meta, on=["parent_asin"], how="inner")
df = pd.merge(df_reviews, df_meta, on=["parent_asin"], how="inner")

In [4]:
df = df.sample(frac=0.01)

In [5]:
nlp = spacy.load('en_core_web_sm')

def clean_text(text):
    doc = nlp(text)
    cleaned_tokens = [token.lemma_.lower() for token in doc if not token.is_stop and not token.is_punct and token.is_alpha]
    return " ".join(cleaned_tokens)

# clean_text_udf = udf(clean_text, StringType())
# df = df.withColumn('cleaned_text', clean_text_udf(df['text']))

In [6]:
def get_sentiment(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

# sentiment_udf = udf(get_sentiment, FloatType())
# df = df.withColumn('sentiment', sentiment_udf(df['cleaned_text']))

In [7]:
# df['cleaned_text'] = df['text'].apply(clean_text)

In [8]:
df['sentiment'] = df['text'].apply(get_sentiment)

In [16]:
px.histogram(df, x='sentiment', nbins=50, title="Sentiment scores histogram").show()

In [29]:
px.violin(df, x='rating', y='sentiment', title="Violin Plot of Sentiment by Rating", box=True).show()

In [26]:
df.loc[(df["rating"] == 5) & (df["sentiment"] == -1), ["text"]].values[0][0]

'Women Go CRAZY!! When I Walk By... Luv The  Fragrance!'